In [1]:
# bibliotecas
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import re
import requests
import base64
import os

Como uma maneira de economizar recursos de memória, a visibilidade do navegador pode ser desativada durante a execução do script. O script abaixo mostra como pode ser realizado essa configuração para o webdriver do Firefox.

In [ ]:
# define a visibilidade do navegador durante a execução do script. 
# options = Options()
# options.add_argument('-headless')

A criação de um diretório para armazenar os arquivos .xml baixados visa melhorar a organização do projeto. O nome do diretório corresponde ao CNPJ do Fundo de Investimento Imobiliário (FII).

In [3]:
cnpj = '11728688000147'
raw = f'../src/data/raw/'
os.mkdir(f'{raw}{cnpj}')

A função abaixo acessa o endereço web do FundosNet e utiliza o CNPJ para especificar o domínio que contêm os documentos do FII. Definiu-se o tempo de 2 segundos para que a página carregasse por completo.

In [9]:
def acessa_pagina_web(cnpj):
    navegador = webdriver.Firefox()
    URL = 'https://fnet.bmfbovespa.com.br/fnet/publico/abrirGerenciadorDocumentosCVM?cnpjFundo='
    navegador.get(''.join([URL, cnpj]))
    time.sleep(2)
    return navegador

In [ ]:
navegador = acessa_pagina_web(cnpj)

Na página web acessada há a possibilidade de filtrar os documentos desejados conforme características específicas. A função procura o id do botão e efetua o clique sobre ele.


In [ ]:
def pressiona_botao_exibir_filtros(id):
    exibir_filtros = navegador.find_element(By.ID, id)
    exibir_filtros.click()
    time.sleep(2)

In [ ]:
pressiona_botao_exibir_filtros('showFiltros')

Uma janela suspensa aparecerá com várias opções de filtragem. As que nos interessa são Categoria, Tipo e Situação. Essas opções de filtragem são composta de elementos dos tipos dropdown, campo de texto e botão. Um clique será executado em cada dropdown para que seja mostrado o campo de texto e suas opções. Em vez de clicar sobre a opção deseja, optou-se por informar o seu nome no campo de texto. Um clique é efetuado no botão Filtrar para executar a filtragem dos documentos.

In [ ]:
def pressiona_dropdown(id):
    dropdown = navegador.find_element(By.ID, id)
    dropdown.click()
    time.sleep(2)

def informa_valor_campo_texto(id, categoria):
    campo_texto = navegador.find_element(By.ID, id)
    campo_texto.send_keys(categoria, Keys.ENTER)
    time.sleep(2)

def pressiona_botao_filtrar(id):
    filtrar = navegador.find_element(By.ID, id)
    filtrar.click()
    time.sleep(2)

In [ ]:
pressiona_dropdown('select2-chosen-2')  # Categoria
informa_valor_campo_texto('s2id_autogen2_search', 'Informes Periódicos')

pressiona_dropdown('select2-chosen-3')  # Tipo
informa_valor_campo_texto('s2id_autogen3_search', 'Informe Mensal Estruturado')

pressiona_dropdown('select2-chosen-5')  # Situação
informa_valor_campo_texto('s2id_autogen5_search', 'Ativo')


Após a aplicação de todos os filtros, é possível ainda modificar a quantidade de registros que podem ser visualizados em uma única página. A função abaixo procura pelo o elemento correspondente à modificação da visibilidade de registros e redefine para a quantidade de registros desejados.

In [ ]:
def modifica_numero_registros_visiveis(num, nome):
    mostrar_registros = navegador.find_element(By.NAME, nome)
    registros = Select(mostrar_registros)
    registros.select_by_visible_text(str(num))
    time.sleep(2)

In [ ]:
pressiona_botao_filtrar('filtrar')
modifica_numero_registros_visiveis(100, 'tblDocumentosEnviados_length')

A página web no seu estado atual disponibiliza a visualização e o download dos documentos. A função obtém todos os links disponíveis na página.

In [ ]:
def obtem_links(atributo):
    todos_links = navegador.find_elements(By.TAG_NAME, 'a')
    links = []
    for link in todos_links:
        links.append(link.get_attribute(atributo))
    return links

In [ ]:
links = obtem_links('href')

Com os links armazenados em uma lista, não é mais necessário manter o navegador aberto. Para economizar espação de memória ele pode ser finalizados.

In [ ]:
navegador.quit()

Alguns elementos da lista de link não possuem um tipo definido. Estes serão removidos. Além destes, serão removidos também link não pertinentes a downlods.

In [ ]:
def remove_tipo_indefinido(lista):
    links_definidos = [link for link in lista if link is not None]
    return links_definidos

def remove_links_invalidos(lista):
    nome_filtro = 'downloadDocumento'
    links = [link for link in lista if nome_filtro in link]
    return links

In [ ]:
links = remove_tipo_indefinido(links)
links_validos = remove_links_invalidos(links)

Por fim, a função abaixo executar os downloads dos informes de rendimento estruturados (no formato xml)a partir da lista de links obtidas anteriormente.

In [ ]:
def download_arquivo(caminho):
    for link in links_validos:
        doc = requests.get(link, allow_redirects=True, verify=True)
        nome = re.findall('filename="(.+)"', doc.headers.get('content-disposition'))[0]
        diretorio = ''.join([caminho, nome])

        with open(diretorio, 'wb') as arquivo:
            arquivo.write(base64.b64decode(doc.content))
        time.sleep(0.5)

In [ ]:
download_arquivo(f'{raw}{cnpj}/')

In [ ]:
print('Download dos arquivos xml concluído')

In [ ]:
import os